<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, roc_auc_score, precision_score, recall_score, f1_score, \
    confusion_matrix, matthews_corrcoef, precision_recall_fscore_support, accuracy_score
import random
import xgboost as xgb
import sys

In [ ]:
dataset = 'geo'
label_type = 'gender'

# load data
path = 'data/' + dataset + '/'
X = np.loadtxt(path + 'X_' + label_type + '.txt', delimiter='\t')
y = np.loadtxt(path + 'y_' + label_type + '.txt', delimiter='\t')
print(X.shape, y.shape)
print(np.unique(y))

In [ ]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
# initialize classifier
clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0)
# fit model
clf.fit(X_train, y_train)

In [ ]:
y_train_pred = clf.predict(X_train)
acc = accuracy_score(y_train, y_train_pred)

p, r, f1, s = precision_recall_fscore_support(y_train, y_train_pred, average='weighted', zero_division=1)
print('Training ACC={:.4f} PS={:.4f} RC={:.4f} F1={:.4f}'.format(acc, p, r, f1))
cm = confusion_matrix(y_train, y_train_pred)
print(cm)

In [ ]:
y_test_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_test_pred)

p, r, f1, s = precision_recall_fscore_support(y_test, y_test_pred, average='weighted', zero_division=1)
print('Testing ACC={:.4f} PS={:.4f} RC={:.4f} F1={:.4f}'.format(acc, p, r, f1))
cm = confusion_matrix(y_test, y_test_pred)
print(cm)

In [ ]:
# 5-fold Cross Validation
method = 'RF'

if method == 'XGB':
    n_est_list = [2, 5, 10]
    max_depth_list = [3, 4, 5]
elif method == 'RF':
    n_est_list = [2, 5, 10]
    max_depth_list = [5, 10]

def cv(X, y, classifier, n_est=10, max_dep=2):
    skf = StratifiedKFold(n_splits=5)
    f = 1
    OUT = []
    for train_index, test_index in skf.split(X, y):
        X_trn, X_val = X[train_index], X[test_index]
        y_trn, y_val = y[train_index], y[test_index]

        if classifier == 'RF':
            clf = RandomForestClassifier(n_estimators=n_est, max_depth=max_dep, min_samples_split=2, random_state=0)
        elif classifier =='XGB':
            if len(np.unique(y)) > 2:
                clf = xgb.XGBClassifier(objective="multi:softprob", n_estimators=n_est, max_depth=max_dep, random_state=0)
            else:
                clf = xgb.XGBClassifier(objective="binary:logistic", n_estimators=n_est, max_depth=max_dep, random_state=0)

        clf.fit(X_trn, y_trn)

        y_trn_pred = clf.predict(X_trn)
        acc = accuracy_score(y_trn, y_trn_pred)
        p, r, f1, s = precision_recall_fscore_support(y_trn, y_trn_pred, average='weighted', zero_division=1)
        mcc = matthews_corrcoef(y_trn, y_trn_pred)
        print('fold {:d} Training ACC={:.2f} PS={:.2f} RC={:.2f} F1={:.2f} MCC={:.2f}'.format(f, acc, p, r, f1, mcc))

        y_val_pred = clf.predict(X_val)
        acc = accuracy_score(y_val, y_val_pred)
        p, r, f1, s = precision_recall_fscore_support(y_val, y_val_pred, average='weighted', zero_division=1)
        mcc = matthews_corrcoef(y_val, y_val_pred)
        print('fold {:d} Testing ACC={:.2f} PS={:.2f} RC={:.2f} F1={:.2f} MCC={:.2f}'.format(f, acc, p, r, f1, mcc))
        print(confusion_matrix(y_val, y_val_pred))
        OUT.append(np.array([acc, p, r, f1, mcc]))
        f += 1

    OUT = np.array(OUT)
    p_avg = np.mean(OUT, axis=0)
    p_std = np.std(OUT, axis=0)
    print('5-fold Average:')
    print('ACC={:.3f} PS={:.3f} RC={:.3f} F1={:.3f} MCC={:.3f}'.format(p_avg[0], p_avg[1], p_avg[2], p_avg[3], p_avg[4]))
    print('ACC={:.3f} PS={:.3f} RC={:.3f} F1={:.3f} MCC={:.3f}'.format(p_std[0], p_std[1], p_std[2], p_std[3], p_std[4]))
    return p_avg

result = []
for n_est in n_est_list:
    for max_d in max_depth_list:
        print('Processing ' + method + ' n_estimators=' + str(n_est) + 'max_depth=' + str(max_d))
        p_avg = cv(X, y, 'XGB', n_est=n_est, max_dep=max_d)
        result.append(p_avg)
result = np.array(result)
print(result)


In [ ]:
from sklearn.model_selection import GridSearchCV

def evaluate(model, x_test, y_test):
    y_pred = model.predict(x_test)
    acc = accuracy_score(y_test, y_pred)
    return acc

param_grid = {
    'max_depth': [2, 10],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4],
    'min_samples_split': [2, 5],
    'n_estimators': [2, 5, 10, 50]
}

# Create a model
clf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = clf, param_grid = param_grid, cv = 5, n_jobs=-1, verbose = 2)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

grid_search.fit(X_train, y_train)
print('The best parameter found using Grid Search:')
print(grid_search.best_params_)

best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, X_test, y_test)
print('The accuracy using best found parmeters {:.4f}'.format(grid_accuracy))